Bai1

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [ ]:
file_test = pd.read_csv(r'/content/winequality-white.csv', sep = ';')
print(f'rows {file_test.shape[0]}')
print(f'columns {file_test.shape[1]}')
print(f"cac thuoc tinh {file_test.columns}")
print(f"gia tri cac nhan {file_test['quality'].unique()}")

rows 4898
columns 12
cac thuoc tinh Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')
gia tri cac nhan [6 5 7 8 4 3 9]


In [ ]:
kf = KFold(n_splits=50, shuffle=True, random_state=42)
num = file_test.shape[0]
test_size = num // 50
train_size = num - test_size
print(f"sl ptu trong tap test: {test_size}")
print(f"sl ptu trong tap train: {train_size}")

sl ptu trong tap test: 97
sl ptu trong tap train: 4801


In [ ]:
accuracies = []
for train_index, test_index in kf.split(file_test.drop(columns=['quality'])):
    X_train = file_test.drop(columns=['quality']).iloc[train_index]
    X_test = file_test.drop(columns=['quality']).iloc[test_index]
    y_train = file_test['quality'].iloc[train_index]
    y_test = file_test['quality'].iloc[test_index]
    model = DecisionTreeClassifier(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

print(f"accuaracy tb: {sum(accuracies) / len(accuracies):.4f}")

accuaracy tb: 0.6323


In [ ]:
X = file_test.drop(columns=['quality'])
y = file_test['quality']

for fold_idx, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model = DecisionTreeClassifier(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    if fold_idx == 50 - 1:
        report = classification_report(y_test, y_pred, output_dict=True)
        print(classification_report(y_test, y_pred))
        precision_per_class = {k: v['precision'] for k, v in report.items() if k.isdigit()}
        print("do chinh xac cho tung phan lop")
        for label, precision in precision_per_class.items():
            print(f"phan lop{label}: {precision:.4f}")


              precision    recall  f1-score   support

           4       0.00      0.00      0.00         5
           5       0.70      0.63      0.67        30
           6       0.70      0.71      0.71        42
           7       0.59      0.76      0.67        17
           8       0.67      0.67      0.67         3

    accuracy                           0.66        97
   macro avg       0.53      0.56      0.54        97
weighted avg       0.64      0.66      0.65        97

do chinh xac cho tung phan lop
phan lop4: 0.0000
phan lop5: 0.7037
phan lop6: 0.6977
phan lop7: 0.5909
phan lop8: 0.6667


In [ ]:
average_accuracy = sum(accuracies) / len(accuracies)

print(f"do chinh xac tb sau 50 lan lap {round(average_accuracy, 4)}", )

do chinh xac tb sau 50 lan lap 0.6323


In [ ]:
K = 60
kf = KFold(n_splits=K, shuffle=True, random_state=42)
accuracies_dt = []
accuracies_knn = []
accuracies_nb = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    dt_model = DecisionTreeClassifier(random_state=42)
    dt_model.fit(X_train, y_train)
    y_pred_dt = dt_model.predict(X_test)
    accuracies_dt.append(accuracy_score(y_test, y_pred_dt))


    knn_model = KNeighborsClassifier(n_neighbors=5)  # Sử dụng K=5 mặc định
    knn_model.fit(X_train, y_train)
    y_pred_knn = knn_model.predict(X_test)
    accuracies_knn.append(accuracy_score(y_test, y_pred_knn))


    nb_model = GaussianNB()
    nb_model.fit(X_train, y_train)
    y_pred_nb = nb_model.predict(X_test)
    accuracies_nb.append(accuracy_score(y_test, y_pred_nb))

avg_accuracy_dt = sum(accuracies_dt) / len(accuracies_dt)
avg_accuracy_knn = sum(accuracies_knn) / len(accuracies_knn)
avg_accuracy_nb = sum(accuracies_nb) / len(accuracies_nb)

print(f"do chinh xac tb cho dt: {avg_accuracy_dt:.4f}")
print(f"do chinh xac tb cho knn: {avg_accuracy_knn:.4f}")
print(f"do chinh xac tb cho bayes: {avg_accuracy_nb}")


do chinh xac tb cho dt: 0.6364
do chinh xac tb cho knn: 0.4961
do chinh xac tb cho bayes: 0.4424721469436917


Bai 2

In [ ]:

data = {
    "Chiều cao": [180, 167, 136, 174, 141],
    "Độ dài tóc": [15, 42, 35, 15, 28],
    "Giọng nói": [0, 1, 1, 0, 1],
    "Nhãn": [0, 1, 1, 0, 1]
}

df = pd.DataFrame(data)

X = df[["Chiều cao", "Độ dài tóc", "Giọng nói"]]
y = df["Nhãn"]

model = DecisionTreeClassifier(criterion="entropy", random_state=42)
model.fit(X, y)

tree_rules = export_text(model, feature_names=list(X.columns))
print(tree_rules)


|--- Độ dài tóc <= 21.50
|   |--- class: 0
|--- Độ dài tóc >  21.50
|   |--- class: 1



In [ ]:
new_data = pd.DataFrame({"Chiều cao": [135], "Độ dài tóc": [39], "Giọng nói": [1]})
prediction = model.predict(new_data)
print(f"Phần tử mới được dự đoán là: {prediction[0]}")


Phần tử mới được dự đoán là: 1
